# Criticality in Lenia: Edge of Chaos Analysis

This notebook analyzes the phase diagram and criticality signatures in Lenia.

## Hypothesis

Life-like patterns emerge at a **critical phase transition** between ordered (dead) and chaotic (explosive) dynamics.

### Predictions:
1. Lyapunov exponent λ → 0 at the critical boundary
2. Correlation length ξ → ∞ (power-law correlations)
3. Maximum complexity/entropy at criticality

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.simulation import LeniaSimulation, LeniaConfig
from src.metrics import LyapunovEstimator, SpatialAnalyzer, InformationMetrics
from src.experiment import ExperimentRunner, ExperimentConfig, Experiment
from src.analysis import (
    setup_publication_style,
    plot_phase_diagram,
    plot_lyapunov_diagram,
    plot_criticality_signatures,
    plot_critical_line_analysis,
    summarize_results,
    find_critical_points,
)

setup_publication_style()
%matplotlib inline

## 1. Quick Visualization: Single Simulation

In [ ]:
# Test a single simulation
config = LeniaConfig(
    grid_size=128,
    mu=0.15,
    sigma=0.015,
    seed=42
)

sim = LeniaSimulation(config)

# Run and visualize
fig, axes = plt.subplots(1, 4, figsize=(14, 3.5))

for i, t in enumerate([0, 50, 150, 300]):
    sim.run(t if i == 0 else (t - [0, 50, 150, 300][i-1]))
    axes[i].imshow(sim.world, cmap='viridis', vmin=0, vmax=1)
    axes[i].set_title(f't = {t}')
    axes[i].axis('off')

plt.suptitle(f'Lenia Evolution (μ={config.mu}, σ={config.sigma})')
plt.tight_layout()

## 2. Run Phase Diagram Experiment

Sweep the (μ, σ) parameter space and classify outcomes.

In [ ]:
# Run experiment (adjust resolution and workers for your system)
# For quick test: resolution=15, for publication: resolution=40+

df = ExperimentRunner.phase_diagram(
    name='notebook_phase',
    resolution=20,  # 20x20 = 400 points
    grid_size=128,
    n_workers=4,
    compute_lyapunov=True,
    lyapunov_trials=2,  # Increase to 5 for better statistics
)

print(f"\nCompleted {len(df)} points")
df.head()

## 3. Phase Diagram Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Classification
plot_phase_diagram(df, ax=axes[0], title='Phase Classification')

# Lyapunov
plot_lyapunov_diagram(df, ax=axes[1])

plt.tight_layout()
plt.savefig('../figures/phase_diagram.png', dpi=150)

## 4. Criticality Signatures

Multi-panel figure showing all criticality indicators.

In [ ]:
fig = plot_criticality_signatures(df)
plt.savefig('../figures/criticality_signatures.png', dpi=150)

## 5. Finding the Critical Line

Identify parameter combinations where λ ≈ 0.

In [ ]:
critical_points = find_critical_points(df, lyapunov_threshold=0.02)
print(f"Found {len(critical_points)} near-critical points:")
critical_points[['mu', 'sigma', 'lyapunov', 'classification', 'entropy']].head(10)

## 6. Statistical Summary

In [ ]:
summary = summarize_results(df)

print("="*50)
print("EXPERIMENT SUMMARY")
print("="*50)

print(f"\nTotal points: {summary['total_points']}")

print("\nPhase counts:")
for phase, count in summary['phase_counts'].items():
    pct = count / summary['total_points'] * 100
    print(f"  {phase}: {count} ({pct:.1f}%)")

if 'lyapunov' in summary:
    print(f"\nLyapunov exponent:")
    print(f"  Mean: {summary['lyapunov']['mean']:.4f}")
    print(f"  Std: {summary['lyapunov']['std']:.4f}")
    print(f"  Range: [{summary['lyapunov']['min']:.4f}, {summary['lyapunov']['max']:.4f}]")
    print(f"  Near-critical (|λ|<0.01): {summary['lyapunov']['n_critical']}")

## 7. Lyapunov vs Classification

Verify that Lyapunov exponent correctly predicts phase.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

# Box plot of Lyapunov by classification
df_valid = df[df['lyapunov'].notna()].copy()

order = ['dead', 'stable', 'oscillating', 'chaotic', 'explosive']
colors = ['#1a1a2e', '#2a9d8f', '#457b9d', '#f4a261', '#e63946']

sns.boxplot(data=df_valid, x='classification', y='lyapunov', 
            order=order, palette=colors, ax=ax)

ax.axhline(0, color='red', linestyle='--', alpha=0.7, label='λ=0 (critical)')
ax.axhspan(-0.01, 0.01, alpha=0.2, color='green', label='Critical zone')

ax.set_xlabel('Phase Classification')
ax.set_ylabel('Lyapunov Exponent (λ)')
ax.set_title('Lyapunov Exponent by Phase')
ax.legend()

plt.tight_layout()
plt.savefig('../figures/lyapunov_by_phase.png', dpi=150)

## 8. Correlation Analysis

Compare correlation functions across phases.

In [ ]:
# Sample one point from each phase and analyze correlations
analyzer = SpatialAnalyzer()

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

test_points = [
    (0.10, 0.008, 'Dead (ordered)'),
    (0.15, 0.015, 'Stable (critical?)'),
    (0.12, 0.010, 'Chaotic'),
]

for i, (mu, sigma, label) in enumerate(test_points):
    config = LeniaConfig(grid_size=128, mu=mu, sigma=sigma, seed=42)
    sim = LeniaSimulation(config)
    sim.run(300)
    
    r, C = analyzer.correlation_function(sim.world)
    result = analyzer.fit_correlation(r, C)
    
    axes[i].loglog(r[r>0], np.abs(C[r>0]), 'o-', markersize=3)
    axes[i].set_xlabel('Distance r')
    axes[i].set_ylabel('|C(r)|')
    axes[i].set_title(f'{label}\nFit: {result.fit_type}')

plt.tight_layout()
plt.savefig('../figures/correlation_comparison.png', dpi=150)

## 9. Detailed Critical Line Analysis

High-resolution scan along the critical boundary.

In [ ]:
# Find best mu from phase diagram
if len(critical_points) > 0:
    best_mu = critical_points['mu'].mode().values[0]
else:
    best_mu = 0.15

print(f"Scanning critical line at μ = {best_mu:.3f}")

# High-resolution scan
df_line = ExperimentRunner.criticality_line(
    mu_center=best_mu,
    sigma_range=(0.008, 0.028),
    resolution=30,
    grid_size=128,
    n_workers=4,
)

In [ ]:
fig = plot_critical_line_analysis(df_line)
plt.savefig('../figures/critical_line_analysis.png', dpi=150)

## 10. Conclusions

Based on the analysis:

1. **Phase Structure**: The (μ, σ) parameter space shows clear phase transitions between dead, stable, oscillating, chaotic, and explosive regimes.

2. **Lyapunov Exponent**: 
   - Ordered (dead) states: λ < 0 (perturbations decay)
   - Critical boundary: λ ≈ 0 (marginal stability)
   - Chaotic states: λ > 0 (exponential divergence)

3. **Correlation Structure**:
   - At criticality: power-law correlations (scale-free)
   - Off-critical: exponential decay with finite correlation length

4. **Life-like patterns** emerge precisely at the edge of chaos, consistent with the criticality hypothesis.

### Future Work
- Map complete critical line in (μ, σ, R) space
- Measure critical exponents for universality class
- Compare with Game of Life and other CA

In [ ]:
# Save all results
df.to_csv('../experiments/final_results.csv', index=False)
print("Results saved!")